# Motor de Investigación

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [12]:
# %pip install langchain langchain_community langchain_openai duckduckgo-search

In [13]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

## Búsqueda en la web

Pasos a realizar:
1. Crear una herramienta de búsqueda en la web.
2. Crear una herramienta de web scraping.
3. Crear los prompts.
4. Implementar la arquitectura.

### Paso 1: Crear una herramienta de búsqueda en la web

In [14]:
# Herramienta de búsqueda en la web

from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from typing import List
 
def web_search(web_query: str, num_results: int) -> List[str]:
    return [r["link"] for r in DuckDuckGoSearchAPIWrapper().results(web_query, num_results)]

In [15]:
# Hagamos una prueba del buscador
 
result = web_search(web_query="Cuántos balones de oro tiene Messi?", num_results=5)
print(result)

['https://www.ole.com.ar/messi/8-balones-oro-messi-titulos-goles-asistencias_0_wY0tK0XVIf.html', 'https://www.dazn.com/es-ES/news/fútbol/cuantos-balones-oro-leo-messi/lqeidd5kuq1111h3n3zxixkyu', 'https://cnnespanol.cnn.com/2023/10/30/lionel-messi-cuantos-balones-de-oro-ganado-orix/', 'https://www.sport.es/es/noticias/balon-oro/balones-oro-messi-total-12264298', 'https://tn.com.ar/deportes/futbol/2023/10/30/cuantos-balones-de-oro-tiene-lionel-messi-el-mejor-de-la-historia/']


Nota 
--- 
LangChain también ofrece otros wrappers para motores de búsqueda web, como **TavilySearchResults** y **GoogleSearchAPIWrapper**. Pero ambos requieren una clave API, por practicidad decidimos usar **DuckDuckGoSearchAPIWrapper**.

### Paso 2: Crear una herramienta de web scraping

In [16]:
# Herramienta de web scraping

import requests
from bs4 import BeautifulSoup
 
def web_scrape(url: str) -> str:
    try:
        response = requests.get(url)
 
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            page_text = soup.get_text(separator=" ", strip=True)
 
            return page_text
        else:
            return f"No se pudo recuperar la página web: Código {response.status_code}"
    except Exception as e:
        print(e)
        return f"No se pudo recuperar la página web: {e}"

In [17]:
# Vamos a probar la herramienta de web scraping

result = web_scrape('https://www.sport.es/es/noticias/balon-oro/balones-oro-messi-total-12264298')
print(result)

¿Cuántos Balones de Oro tiene Messi en total? Temas Real Madrid - Borussia Dortmund Girona Slovan Bratislava Champions hoy Clasificación Champions Madrid - Dortmund horario Bronca Bernabéu Raphinha Nico Williams Topuria - Holloway horario Xavi United Barcelona Youth League Balón de Oro Ceciarmy NBA 24-25 Entradas Clásico Mundial de Clubes Clasificación Euroliga Gonzalo Bernardos Programación DAZN Clasificación Grupo España Últimas noticias Clasificación LaLiga Clasificación Champions F Formato Champions Dinero Champions Worlds LoL 2024 Fichajes Barça Dónde ver LaLiga Clasificación F1 Clasificación MotoGP Próxima carrera F1 Suscripción sport ÚLTIMA HORA Barça Barça Clásico Fichajes Última hora Femenino Baloncesto Balonmano Fútbol Sala Hockey Cantera Real Madrid Real Madrid Clásico Calendario Fichajes Última hora Plantilla Vídeos Real Madrid Galerías Real Madrid Fútbol Fútbol LaLiga EA Sports Liga F Fútbol Internacional LaLiga Hypermotion Copa del Rey Selección Champions League Europa Le

### Paso 3: Crear los prompts

In [20]:
from langchain.prompts import PromptTemplate

instrucciones_asistente_seleccion = """
Eres experto en asignar una pregunta de investigación al asistente de investigación correcto.
Hay varios asistentes de investigación disponibles, cada uno especializado en un área de experiencia.
Cada asistente está identificado por un tipo específico y tiene instrucciones específicas para llevar a cabo la investigación.

Cómo seleccionar el asistente correcto: debes seleccionar el asistente relevante dependiendo del tema de la pregunta, que debe coincidir con el área de experiencia del asistente.

------
Aquí tienes algunos ejemplos de cómo devolver la información correcta del asistente, según la pregunta realizada.

Ejemplos:
Pregunta: "¿Debería invertir en títulos de cobertura?"
Respuesta: 
{{
    "tipo_asistente": "Asistente analista financiero",
    "instrucciones_asistente": "Eres un asistente de inteligencia artificial experto en análisis financiero. Tu objetivo principal es elaborar informes financieros completos, perspicaces, imparciales y organizados metódicamente basados en los datos y tendencias proporcionados.",
    "pregunta_usuario": {pregunta_usuario}
}}
Pregunta: "¿Cuáles son los sitios más interesantes en Río de Janeiro?"
Respuesta: 
{{
    "tipo_asistente": "Asistente de guía turístico",
    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",
    "pregunta_usuario": "{pregunta_usuario}"
}}

Pregunta: "¿Es Messi un buen jugador de fútbol?"
Respuesta: 
{{
    "tipo_asistente": "Asistente experto en deportes",
    "instrucciones_asistente": "Eres un asistente de inteligencia artificial especializado en deportes. Tu propósito principal es redactar informes deportivos atractivos, perspicaces, imparciales y bien estructurados sobre personalidades deportivas o eventos deportivos, incluyendo detalles factuales, estadísticas y análisis.",
    "pregunta_usuario": "{pregunta_usuario}"
}}

------
Ahora que has entendido todo lo anterior, selecciona el asistente de investigación correcto para la siguiente pregunta.
Pregunta: {pregunta_usuario}
Respuesta:

""" 

plantilla_asistente_seleccion = PromptTemplate.from_template(
    template=instrucciones_asistente_seleccion
)


In [21]:
instrucciones_busqueda_web = """
{instrucciones_asistente}

Escribe {num_consultas_busqueda} consultas de búsqueda web para recopilar la mayor cantidad de información posible 
sobre la siguiente pregunta: {pregunta_usuario}. Tu objetivo es escribir un informe basado en la información que encuentres.
Debes responder con una lista de consultas como consulta1, consulta2, consulta3 en el siguiente formato: 
[
    {{"consulta_busqueda": "consulta1", "pregunta_usuario": "{pregunta_usuario}" }},
    {{"consulta_busqueda": "consulta2", "pregunta_usuario": "{pregunta_usuario}" }},
    {{"consulta_busqueda": "consulta3", "pregunta_usuario": "{pregunta_usuario}" }}
]
"""

plantilla_busqueda_web = PromptTemplate.from_template(
    template=instrucciones_busqueda_web
)


In [22]:
instrucciones_resumen = """
Lee el siguiente texto:
Texto: {texto_resultado_busqueda}

-----------

Usando el texto anterior, responde brevemente la siguiente pregunta.
Pregunta: {consulta_busqueda}

Si no puedes responder la pregunta anterior usando el texto proporcionado, simplemente resume el texto.

Incluye toda la información factual, números, estadísticas, etc., si están disponibles.
"""

plantilla_resumen = PromptTemplate.from_template(
    template=instrucciones_resumen
)


In [50]:
# Prompt de informe de investigación adaptado de https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py

instrucciones_informe_investigacion = """
Eres un asistente de investigación de pensamiento crítico impulsado por IA. Tu único propósito es escribir informes bien redactados, aclamados críticamente, objetivos y estructurados sobre el texto proporcionado.

Información: 
--------
{resumen_investigacion}
--------

Usando la información anterior, responde la siguiente pregunta o tema: "{pregunta_usuario}" en un informe detallado -- \
El informe debe centrarse en la respuesta a la pregunta, estar bien estructurado, ser informativo, \
profundo, con hechos y cifras si están disponibles, y tener un mínimo de 1,200 palabras.

Debes esforzarte por escribir el informe lo más extenso posible utilizando toda la información relevante y necesaria proporcionada.
Debes escribir el informe con sintaxis de markdown.
DEBES determinar tu propia opinión concreta y válida basada en la información proporcionada. NO infieras conclusiones generales y sin sentido.
Escribe todas las URLs de las fuentes utilizadas al final del informe y asegúrate de no agregar fuentes duplicadas, solo una referencia por cada una.
Debes escribir el informe en formato APA."""
 
plantilla_informe_investigacion = PromptTemplate.from_template(
    template=instrucciones_informe_investigacion
)


### Paso 4: Implementar la arquitectura

In [24]:
# Creemos la instancia del LLM
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    temperature=0.6,
)

In [26]:
# Constantes

NUMERO_CONSULTAS_BUSQUEDA = 2
NUMERO_RESULTADOS_POR_CONSULTA = 3
MAXIMO_CARACTERES_TEXTO_RESULTADO = 10000

In [27]:
# El único input que necesitamos es la pregunta del usuario

pregunta = "Qué se puede hacer en la Isla de Margarita?"

El primer paso de la arquitectura es ejecutar el prompt del LLM para determinar el asistente de investigación correcto basadas en la pregunta de investigación del usuario

In [29]:
prompt_asistente_seleccion = plantilla_asistente_seleccion.format(pregunta_usuario=pregunta)

instrucciones_asistente = llm.invoke(prompt_asistente_seleccion)

# Veamos el resultado
print(instrucciones_asistente.content)

{
    "tipo_asistente": "Asistente de guía turístico",
    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",
    "pregunta_usuario": "Qué se puede hacer en la Isla de Margarita?"
}


In [31]:
# Convertimos el resultado en un objeto
import json

dict_instrucciones_asistente = json.loads(instrucciones_asistente.content)
dict_instrucciones_asistente

{'tipo_asistente': 'Asistente de guía turístico',
 'instrucciones_asistente': 'Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.',
 'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'}

El siguiente paso en la arquitectura es ejecutar el prompt de búsqueda web para recopilar información relevante sobre la pregunta de investigación del usuario

In [32]:
prompt_busqueda_web = plantilla_busqueda_web.format(instrucciones_asistente=dict_instrucciones_asistente['instrucciones_asistente'],
                                                    num_consultas_busqueda=NUMERO_CONSULTAS_BUSQUEDA,
                                                    pregunta_usuario=dict_instrucciones_asistente['pregunta_usuario'])
consultas_busqueda_web = llm.invoke(prompt_busqueda_web)
lista_consultas_busqueda_web = json.loads(consultas_busqueda_web.content)
lista_consultas_busqueda_web


[{'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'},
 {'consulta_busqueda': 'atracciones principales Isla de Margarita',
  'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'},
 {'consulta_busqueda': 'guía de viaje Isla de Margarita',
  'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'}]

In [33]:
# Vamos a realizar la búsqueda web
resultados_busquedas = [{'urls_resultado': web_search(web_query=wq['consulta_busqueda'], 
                                     num_results=NUMERO_RESULTADOS_POR_CONSULTA), 
                           'consulta_busqueda': wq['consulta_busqueda']} 
                           for wq in lista_consultas_busqueda_web]


In [34]:
resultados_busquedas

[{'urls_resultado': ['https://www.dviaje.com.ve/margarita/',
   'https://www.tripadvisor.com/Attractions-g316061-Activities-oa30-Margarita_Island_Coastal_Islands_Insular_Region.html',
   'https://www.cosasdeviajes.com/que-ver-en-isla-margarita/'],
  'consulta_busqueda': 'actividades turísticas Isla de Margarita'},
 {'urls_resultado': ['https://www.ldhoteles.com/guia-de-viaje-para-la-isla-de-margarita',
   'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/',
   'https://www.dviaje.com.ve/margarita/'],
  'consulta_busqueda': 'atracciones principales Isla de Margarita'},
 {'urls_resultado': ['https://www.ldhoteles.com/guia-de-viaje-para-la-isla-de-margarita',
   'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/',
   'https://www.dviaje.com.ve/margarita/'],
  'consulta_busqueda': 'guía de viaje Isla de Margarita'}]

Cada consulta de búsqueda web tiene una lista de URLs de resultados. Ahora, necesitamos extraer el texto de cada URL para resumirlo y responder a la pregunta de investigación del usuario.

In [35]:
# Vamos a "aplanar" la lista de resultados de búsqueda

resultados_busquedas_aplanado = []
for qr in resultados_busquedas:
    resultados_busquedas_aplanado.extend([{'consulta_busqueda': qr['consulta_busqueda'], 
                                    'urls_resultado': r
                                    } for r in qr['urls_resultado']])

In [36]:
resultados_busquedas_aplanado

[{'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'urls_resultado': 'https://www.dviaje.com.ve/margarita/'},
 {'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'urls_resultado': 'https://www.tripadvisor.com/Attractions-g316061-Activities-oa30-Margarita_Island_Coastal_Islands_Insular_Region.html'},
 {'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'urls_resultado': 'https://www.cosasdeviajes.com/que-ver-en-isla-margarita/'},
 {'consulta_busqueda': 'atracciones principales Isla de Margarita',
  'urls_resultado': 'https://www.ldhoteles.com/guia-de-viaje-para-la-isla-de-margarita'},
 {'consulta_busqueda': 'atracciones principales Isla de Margarita',
  'urls_resultado': 'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/'},
 {'consulta_busqueda': 'atracciones principales Isla de Margarita',
  'urls_resultado': 'https://www.dviaje.com.ve/margarita/'},
 {'consulta_busqueda': 'guía de viaje Isla de Margarit

In [39]:
# Ahora usaremos la función de web scraping para extraer el texto de cada URL

textos_resultados = [ {'texto_resultado_busqueda': web_scrape(url=re['urls_resultado'])[:MAXIMO_CARACTERES_TEXTO_RESULTADO],
                     'urls_resultado': re['urls_resultado'],
                     'consulta_busqueda': re['consulta_busqueda']}
                   for re in resultados_busquedas_aplanado]

In [40]:
textos_resultados

[{'texto_resultado_busqueda': 'Margarita | La Perla del Caribe ⭐ Dias del sol y playa esperan La Perla del Caribe Margarita Playas hermosas y hoteles todo incluido esperan Enviar un Email La Perla del Caribe Margarita Playas hermosas y hoteles todo incluido esperan Enviar un Email La Isla de Margarita , conocida como La Perla del Caribe , es uno de los destinos turísticos más importantes y encantadores de Venezuela. Ubicada en el mar Caribe, Margarita ofrece una combinación única de playas paradisíacas, una vibrante vida nocturna y una oferta hotelera variada que se adapta a todo tipo de viajeros. Si estás considerando un lugar para tus próximas vacaciones, esta isla tiene todo lo que necesitas para vivir una experiencia inolvidable. ¿Por Qué Elegir Margarita? Margarita es mucho más que un destino de sol y playa. Además de sus paisajes espectaculares, la isla cuenta con una rica historia, cultura local, actividades de aventura y una infraestructura hotelera que abarca desde opciones de

El paso siguiente es resumir cada página. Este resumen mantendrá la consulta y las urls porque las necesitaremos para el informe de investigación.

In [45]:
lista_resumen_texto = []


# Lo hacemos secuencialmente, pero se puede hacer en paralelo
for rt in textos_resultados: 
    prompt_resumen = plantilla_resumen.format(
        texto_resultado_busqueda=rt['texto_resultado_busqueda'], 
        consulta_busqueda=rt['consulta_busqueda'])
    
    respuesta_llm = llm.invoke(prompt_resumen)
    resumen_texto = respuesta_llm.content
 
    lista_resumen_texto.append({'resumen_texto': resumen_texto,
                         'urls_resultado': rt['urls_resultado'],
                         'consulta_busqueda': rt['consulta_busqueda']})

In [46]:
lista_resumen_texto

[{'resumen_texto': 'La Isla de Margarita ofrece una variedad de actividades turísticas para todos los gustos, que incluyen:\n\n1. **Tours por la Isla**: Visitas a la Basílica de Nuestra Señora del Valle y excursiones al Castillo de San Carlos de Borromeo en Pampatar, que ofrece vistas panorámicas de la costa.\n\n2. **Parques Temáticos y Diversión Familiar**: El Parque El Agua, uno de los parques acuáticos más grandes del país, y el Parque Temático Diverland, ideal para los más pequeños.\n\n3. **Deportes Acuáticos**: Actividades como windsurf y kitesurf en Playa El Yaque, así como buceo y snorkeling en sitios organizados por operadores turísticos. También se pueden hacer paseos en catamarán o kayak en La Restinga.\n\n4. **Compras**: Margarita es zona libre de impuestos, especialmente en Porlamar, donde se pueden encontrar productos a precios competitivos.\n\n5. **Vida Nocturna**: Pampatar cuenta con una animada vida nocturna, con numerosos restaurantes, bares y discotecas.\n\n6. **Gastr

Finalmente, el último paso es escribir el informe de investigación utilizando los resúmenes de cada página web.

In [51]:
# Vamos a convertir en string el objeto lista_resumen_texto
resumenes_string = json.dumps(lista_resumen_texto)

# Creamos el prompt para el informe de investigación
prompt_reporte_investigacion = plantilla_informe_investigacion.format(
    resumen_investigacion=resumenes_string,
    pregunta_usuario=pregunta
)

# Generamos el informe de investigación

respuesta_llm = llm.invoke(prompt_reporte_investigacion)

reporte_investigacion = respuesta_llm.content


# Veamos el resultado
print(reporte_investigacion)

# Informe sobre Actividades Turísticas en la Isla de Margarita

## Introducción

La Isla de Margarita, ubicada en el mar Caribe al noreste de Venezuela, es un destino turístico destacado que atrae a visitantes por su clima cálido, playas paradisíacas, y una rica mezcla cultural. Conocida como "La Perla del Caribe", Margarita ofrece una amplia gama de actividades y atracciones que se adaptan a diferentes tipos de viajeros, desde aquellos que buscan aventura hasta los que prefieren el relax y la cultura. Este informe detalla las diversas actividades disponibles en la isla, organizadas en secciones que abarcan desde turismo cultural hasta deportes acuáticos y gastronomía local.

## 1. Actividades Culturales y Turísticas

### 1.1 Tours por la Isla

Una de las actividades más recomendadas en la Isla de Margarita son los tours que permiten a los visitantes explorar su rica historia y cultura. Entre los lugares destacados se encuentran:

- **Basílica de Nuestra Señora del Valle**: Ubicada en 